In [1]:
# Data wrangling
import pandas as pd

# Scientific
import numpy as np

# Machine learning
try:
    #import xgboost as xgb
    from xgboost import XGBRegressor
except:
    !pip install xgboost
    from xgboost import XGBRegressor
    #import xgboost as xgb

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance
from sklearn import preprocessing
    
try:
    import shap
except:
    !pip install shap
    import shap
    
# Graphics
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns # for correlation heatmap

#model loader
import pickle

In [4]:
!pip install xgboost==1.6.2

In [2]:
seed=100

In [6]:
!conda update xgboost / y


PackageNotInstalledError: Package is not installed in prefix.
  prefix: /data/ramonedaj/miniconda3/envs/dsPipeline3
  package name: /




In [4]:
#filename = 'pH_model_2022-09-23_original.sav' 
#filename = 'pH_model_2022-10-17_332.sav'
filename = '56genes_noselection_mae_0.934/pH_model_2022-10-17_56_full.sav'
#filename = '56to38genes/pH_model_2022-10-17_56.sav'
model_perm_subset = pickle.load(open(filename, 'rb'))

In [5]:
#df = pd.read_csv('Acido_neutralo_alkaliphile_ready_model_27genes.csv')
#df = pd.read_csv('Original_data_model_validation.csv')
#df = pd.read_csv('Y_data_model_ph.csv')
#df = pd.read_csv('subset_model_original_noph.csv')
#df = pd.read_csv('Testset_alldatasets_from_scratch_onlygenes_model.csv')
#df = pd.read_csv('Subset_model_Jones2021_99confidence.csv')
#df = pd.read_csv('HighConfidenceOptima_Jones2021_genes_model.csv')
#df = pd.read_csv('Madin_genes_edited_ok.csv')
#df = pd.read_csv('Jones2021_genes_model.csv')
#df = pd.read_csv('Model_genes_phpref_jones2021.csv')
#df = pd.read_csv('2000_genes_GTDB_model.csv')
#df = pd.read_csv('35genes_model.csv')
#df = pd.read_csv('56genes_noselection/56genes_model_including_pHpref.csv')
#df = pd.read_csv('56to38genes/38genes_model_including_pHpref.csv')
df = pd.read_csv('AcidoNeutroValidation/56genes_acido_neutro_model.csv')

In [54]:
#gene_order = ['DPRP','NAD_Gly3P_dh_N','Pro-kuma_activ','CAP_assoc_N',
# 'WXXGXW','NicO','POTRA_2','CsbD',
# 'Lactate_perm','FixH','Phosphoesterase','AA_permease',
# 'BrnA_antitoxin','Oxygenase-NA','ExoD','Lactonase',
# 'Muramidase','ArsB','Peptidase_M32','TPM_phosphatase',
# 'SNase','Endonuclease_5','LppC','Lum_binding',
# 'TraG-D_C','Topoisom_I','Queuosine_synth']

#gene_order = ['pH.preference','DPRP','WXXGXW','Ku','Pro-kuma_activ','Oxygenase-NA','NAD_Gly3P_dh_N','Phytase-like','GST_C_2','PAS','Melibiase_2',
#              'Linocin_M18','ExoD','OFeT_1','NicO','ABATE','MbtH','Xre_MbcA_ParS_C','GTP_CH_N','Phosphoesterase','Ytca','Queuosine_synth','MgtE',
#              'CorA','Ion_trans','CsbD','CitMHS','Topoisom_I','FTHFS','DctP','Nramp','PRK','Peptidase_C10','Cellulase','Ligase_CoA_2']

gene_order = ['pH.preference','WXXGXW','YHS','CsbD','Pro-kuma_activ','Phosphoesterase','OprB','SOUL','MNHE','Lactonase','TctA','PQQ','Big_3_5',
              'Met_gamma_lyase','MgtE','PGI','UvdE','Cytidylate_kin2','CpsB_CapC','HTH_37','AcetylCoA_hyd_C','Glucodextran_N','TelA',
              'Phenol_MetA_deg','MrpF_PhaF','KdpD','HycI','KdpA','MCRA','TFR_dimer','SNase','CytoC_RC','Methyltransf_4','FAD_binding_7',
              'GWxTD_dom','KdpC','HipA_2','DHquinase_I','PhaG_MnhG_YufB']

df = df.reindex(columns=gene_order)
df

,pH.preference,WXXGXW,YHS,CsbD,Pro-kuma_activ,Phosphoesterase,OprB,SOUL,MNHE,Lactonase,...,TFR_dimer,SNase,CytoC_RC,Methyltransf_4,FAD_binding_7,GWxTD_dom,KdpC,HipA_2,DHquinase_I,PhaG_MnhG_YufB
0,3.440042,1,0,1,1,1,1,0,1,1,...,0,0,0,1,1,0,1,0,0,0
1,3.440042,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,1,0,0,0
2,3.440042,1,0,1,1,1,1,0,0,1,...,1,0,0,1,1,1,1,1,1,1
3,3.880140,1,0,1,1,1,1,0,0,1,...,1,0,0,1,1,1,1,1,1,0
4,4.139316,0,1,1,0,1,1,0,1,1,...,0,1,1,1,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,8.501775,0,0,1,0,1,0,0,0,1,...,1,1,0,1,1,1,1,1,0,1
145,8.603554,0,0,1,0,0,0,0,0,1,...,0,1,0,1,0,0,1,1,0,0
146,8.603638,0,1,1,0,0,0,0,1,1,...,0,1,0,1,1,0,0,0,0,1
147,8.749958,0,0,1,0,0,0,0,1,1,...,0,0,0,1,1,0,1,1,0,0


In [6]:
df

,5-FTHF_cyc-lig,AAA_25,AAA_assoc_C,AcetylCoA_hyd_C,ASH,Big_3_5,CitMHS,CpsB_CapC,CpXC,CsbD,...,SOUL,TctA,TelA,TFR_dimer,TPP_enzyme_C,TrbI,UvdE,WXXGXW,YHS,zf-CDGSH
0,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
2,1,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,1,0,1,1,0
3,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
375,0,0,1,0,1,1,0,1,1,1,...,0,0,0,0,1,1,0,1,0,1
376,0,0,1,1,1,1,1,0,1,1,...,0,0,0,0,1,1,0,1,0,1
377,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,1


In [56]:
X, y = df.loc[:, df.columns != 'pH.preference'], df['pH.preference']

In [8]:
X = df
X

,5-FTHF_cyc-lig,AAA_25,AAA_assoc_C,AcetylCoA_hyd_C,ASH,Big_3_5,CitMHS,CpsB_CapC,CpXC,CsbD,...,SOUL,TctA,TelA,TFR_dimer,TPP_enzyme_C,TrbI,UvdE,WXXGXW,YHS,zf-CDGSH
0,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,1
2,1,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,1,0,1,1,0
3,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
375,0,0,1,0,1,1,0,1,1,1,...,0,0,0,0,1,1,0,1,0,1
376,0,0,1,1,1,1,1,0,1,1,...,0,0,0,0,1,1,0,1,0,1
377,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,1


In [9]:
# Predict the model
#pred = model_perm_subset.predict(df)
pred_perm_subset = model_perm_subset.predict(X)

# MAE Computation
#scores_MAE = mean_absolute_error(y, pred_perm_subset)
#scores_MAE = mean_absolute_error(y, pred_perm_subset)

# RMSE Computation
#scores_RMSE = np.sqrt(mean_squared_error(y, pred_perm_subset))
#print("RMSE : % f, MAE : % f" % (scores_RMSE, scores_MAE))

In [10]:
pd.DataFrame(pred_perm_subset)

,0
0,6.642254
1,7.024584
2,6.539738
3,6.155508
4,6.747923
...,...
374,7.052594
375,4.698838
376,4.706218
377,6.781224


In [11]:
#pd.DataFrame(pred_perm_subset).to_csv('Predicted_Madin_Oct2022.csv')
#pd.DataFrame(pred).to_csv('Predicted_original_validation.csv')
#pd.DataFrame(pred_perm_subset).to_csv('Predicted_AcidoNeutro_Oct2022.csv')
#pd.DataFrame(pred_perm_subset).to_csv('Predicted_SubsetOriginal_Oct2022.csv')
#pd.DataFrame(pred_perm_subset).to_csv('Predicted_testset_fromscratch_Oct2022.csv')
#pd.DataFrame(pred_perm_subset).to_csv('332to34genes_MAE_0.945/Predicted_Jones2021_HigConfidenceOptima_95confidence_genomes.csv')
#pd.DataFrame(pred_perm_subset).to_csv('56genes_noselection_MAE_0.934/Predicted_Jones2021_56genes_HigConfidenceOptima_95confidence_genomes_MAE.csv')
#pd.DataFrame(pred_perm_subset).to_csv('56to38genes/Predicted_Jones2021_56to38genes_HigConfidenceOptima_95confidence_genomes_MAE.csv')
pd.DataFrame(pred_perm_subset).to_csv('AcidoNeutroValidation/Predicted_acidoneutro_56genes.csv')

In [4]:
!pip show xgboost

Name: xgboost
Version: 1.6.2
Summary: XGBoost Python Package
Home-page: https://github.com/dmlc/xgboost
Author: 
Author-email: 
License: Apache-2.0
Location: /data/ramonedaj/miniconda3/envs/dsPipeline3/lib/python3.8/site-packages
Requires: scipy, numpy
Required-by: boostaroota
